### Подготовка данных

In [ ]:
import ibis
import pandas as pd

In [2]:
con = ibis.duckdb.connect()

In [3]:
actions = con.read_csv('data/actions.csv')
triggers = con.read_csv('data/triggers.csv')

Делим таблицу на ту, где есть отклик на триггер и где его нет

In [4]:
actions_result_1 = actions.filter(actions['result'] == 1)
actions_result_0 = actions.filter(actions['result'] == 0)

Делаем предобработку каждой полученной таблицы, чтобы добавить к каждому действия пользователя свой триггер

In [ ]:
joined_result_1 = actions_result_1.join(
    triggers,
    predicates=[actions_result_1['guid'] == triggers['guid']],  # Условие для соединения
    how='inner'  # Внутреннее соединение
)

filtered_result_1 = joined_result_1.filter(
    joined_result_1["date"] >= joined_result_1["date_right"]
).order_by([joined_result_1['guid']])

filtered_result_1_order = filtered_result_1.order_by(
    [filtered_result_1["guid"], filtered_result_1["date_right"].desc()]
)

filtered_result_1_order_ = filtered_result_1_order.mutate(
    time_diff=ibis.case()
        .when(filtered_result_1_order['date'] > filtered_result_1_order['date_right'],
              (filtered_result_1_order['date'] - filtered_result_1_order['date_right']).as_unit('s'))
        .else_((filtered_result_1_order['date_right'] - filtered_result_1_order['date']).as_unit('s'))
        .end()
)

closest_dates_1 = filtered_result_1_order_.group_by(['guid', 'date']).aggregate(
    closest_date_right=filtered_result_1_order_['date_right'].argmin('time_diff')
)

final_table_1 = closest_dates_1.join(
    triggers,
    predicates=[
        closest_dates_1['guid'] == triggers['guid'],
        closest_dates_1['closest_date_right'] == triggers['date']
    ]
).select(
    closest_dates_1['guid'],
    closest_dates_1['date'],
    closest_dates_1['closest_date_right'],
    triggers['trigger'],
    triggers['type']
)

final_table_1 = final_table_1.distinct()

In [ ]:
df_1 = final_table_1.execute()
df_1['time_in_minutes'] = (df_1['date'] - df_1['closest_date_right']).dt.total_seconds() / 60
df_1.to_csv('data/trigger_on.csv', index=False)

---

In [47]:
joined_result_0 = actions_result_0.join(
    triggers,
    predicates=[actions_result_0['guid'] == triggers['guid']],  # Условие для соединения
    how='inner'  # Внутреннее соединение
)

filtered_result_0 = joined_result_0.filter(
    joined_result_0["date"] >= joined_result_0["date_right"]
).order_by([joined_result_0['guid']])

filtered_result_0_order = filtered_result_0.order_by(
    [filtered_result_0["guid"], filtered_result_0["date_right"].desc()]
)

filtered_result_0_order_ = filtered_result_0_order.mutate(
    time_diff=ibis.case()
        .when(filtered_result_0_order['date'] > filtered_result_0_order['date_right'],
              (filtered_result_0_order['date'] - filtered_result_0_order['date_right']).as_unit('s'))
        .else_((filtered_result_0_order['date_right'] - filtered_result_0_order['date']).as_unit('s'))
        .end()
)

closest_dates_0 = filtered_result_0_order_.group_by(['guid', 'date']).aggregate(
    closest_date_right=filtered_result_0_order_['date_right'].argmin('time_diff')
)

final_table_0 = closest_dates_0.join(
    triggers,
    predicates=[
        closest_dates_0['guid'] == triggers['guid'],
        closest_dates_0['closest_date_right'] == triggers['date']
    ]
).select(
    closest_dates_0['guid'],
    closest_dates_0['date'],
    closest_dates_0['closest_date_right'],
    triggers['trigger'],
    triggers['type']
)

final_table_0 = final_table_0.distinct()

In [ ]:
df_0 = final_table_0.execute()
df_0['time_in_minutes'] = (df_0['date'] - df_0['closest_date_right']).dt.total_seconds() / 60
df_0.to_csv('data/trigger_off.csv', index=False)

---

### Проведем обработку полученных данных

Создадим новые колонки: выходной/будний день и время суток отклика

In [59]:
def is_weekend(date):
    # Преобразуем строку в дату и проверяем день недели
    return 1 if pd.to_datetime(date).weekday() >= 5 else 0

In [ ]:
triggers_on = pd.read_csv('data/trigger_on.csv')
triggers_on['weekend'] = triggers_on['date'].apply(is_weekend)
triggers_off = pd.read_csv('data/trigger_off.csv')
triggers_off['weekend'] = triggers_off['date'].apply(is_weekend)
triggers_on.to_csv('data/triggers_on_with_weekend.csv', index=False)
triggers_off.to_csv('data/triggers_off_with_weekend.csv', index=False)

In [67]:
def is_morning(date):
    hour = pd.to_datetime(date).hour
    return 1 if 6 <= hour < 10 else 0

def is_afternoon(date):
    hour = pd.to_datetime(date).hour
    return 1 if 10 <= hour < 18 else 0

def is_evening(date):
    hour = pd.to_datetime(date).hour
    return 1 if 18 <= hour < 24 else 0

def is_night(date):
    hour = pd.to_datetime(date).hour
    return 1 if 0 <= hour < 6 else 0

In [68]:
triggers_on = pd.read_csv('data/triggers_on_with_weekend.csv')
triggers_on['morning'] = triggers_on['date'].apply(is_morning)
triggers_on['afternoon'] = triggers_on['date'].apply(is_afternoon)
triggers_on['evening'] = triggers_on['date'].apply(is_evening)
triggers_on['night'] = triggers_on['date'].apply(is_night)
triggers_on.to_csv('data/triggers_on_with_datetime.csv', index=False)

In [69]:
triggers_off = pd.read_csv('data/triggers_off_with_weekend.csv')
triggers_off['morning'] = triggers_off['date'].apply(is_morning)
triggers_off['afternoon'] = triggers_off['date'].apply(is_afternoon)
triggers_off['evening'] = triggers_off['date'].apply(is_evening)
triggers_off['night'] = triggers_off['date'].apply(is_night)
triggers_off.to_csv('data/triggers_off_with_datetime.csv', index=False)

In [73]:
triggers_on['result'] = 1
triggers_off['result'] = 0

new_actions = pd.concat([triggers_on, triggers_off], ignore_index=True)
new_actions.to_csv('data/new_actions.csv', index=False)

Удалим все строки, которые находятся в new_actions, из triggers

In [76]:
new_actions_ = con.read_csv('data/new_actions.csv')
triggers_ = con.read_csv('data/triggers.csv')

In [80]:
filtered_triggers = triggers_.left_join(new_actions_, 
                                      predicates=[triggers_['guid'] == new_actions_['guid'],
                                                  triggers_['date'] == new_actions_['closest_date_right'],
                                                  triggers_['trigger'] == new_actions_['trigger']]) \
                            .filter(new_actions_['guid'].isnull()) 

In [82]:
filtered_triggers.to_csv('data/filtered_triggers.csv')

In [3]:
filtered_triggers = con.read_csv('data/filtered_triggers.csv')
new_actions = con.read_csv('data/new_actions.csv')

In [6]:
final_triggers = filtered_triggers.filter(~filtered_triggers['guid'].isin(new_actions['guid']))

In [8]:
final_triggers.to_csv('data/final_triggers.csv')